<div class="alert alert-block alert-success">
    
# FIT5196 Task 1 in Assessment 1
#### Student Name 1: Animesh Dubey
#### Student ID: 33758484
#### Student Name 2: Ashwin Gururaj
#### Student ID: 33921199

Date: 01 Aug 2024


Environment: Python 3.11.5

Libraries used:
* re
* pandas
* numpy
* json
* os
* datetime
    
</div>

## Import Libraries

In [10]:
# Importing necessary libraries: 're' for regex, 'pandas' for data manipulation, 'numpy' for numerical operations, 'json' for JSON handling, 'os' for OS-level operations, and 'datetime' for date and time management
import re
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime

're': This library allows us to use regular expressions, which are patterns that help us search and manipulate strings based on specific criteria. It's particularly useful for extracting information from semi-structured text data.<br>
'pandas': A powerful data manipulation library that provides data structures like DataFrames, which make it easier to handle and analyze structured data such as tables.<br>
'numpy': This library supports operations on large arrays and matrices, providing a wide array of mathematical functions that are crucial for numerical computations.<br>
'json': This module is used for parsing JSON (JavaScript Object Notation) data, which is a common format for exchanging data between a server and a client.<br>
'os': The os module helps us interact with the operating system, allowing us to manage file paths and directories, making our code more flexible and adaptable to different environments.<br>
'datetime': This module allows us to work with dates and times, which will be important when we need to handle and manipulate timestamp data in our analysis.



## Set Up File Paths

In [11]:
# Get the directory of the current script
script_dir = os.getcwd()

# Define file paths dynamically using os.path.join
excel_file_path = os.path.join(script_dir, 'group100.xlsx')
input_file_paths = [os.path.join(script_dir, f'group100_{i}.txt') for i in range(15)]
csv_output_path = os.path.join(script_dir, 'task1_100.csv')
json_output_path = os.path.join(script_dir, 'task1_100.json')

The script_dir variable is set using os.getcwd(), which dynamically retrieves the directory path where the current script is located. This makes the code more portable, allowing it to correctly reference files relative to the script’s location regardless of where the script is executed.<br>
File paths (excel_file_path, input_file_paths, csv_output_path, json_output_path) are dynamically defined using os.path.join(), ensuring portability across different systems.<br>

## Defining Regular Expression Patterns for Extracting Fields from Text Data

In [12]:
# Define regex patterns for extracting various fields from the text data.
regex_patterns = {
    'user_id': re.compile(r"<\s*(?:UserId|userid|UserId\.?|user_id|user)>\s*(\d+)\s*<", re.IGNORECASE),
    'name': re.compile(r"<\s*(?:Name|username|user_name|user)>\s*(.*?)\s*<", re.IGNORECASE),
    'time': re.compile(r"<\s*(?:time|date|Date|Time)>\s*(\d+)\s*<", re.IGNORECASE),
    'rating': re.compile(r"<\s*(?:rate|rating|Rate|Rating)>\s*(\d+)\s*<", re.IGNORECASE),
    'text': re.compile(r"<\s*(?:Text|review|Review|text|Review_text)>\s*(.*?)\s*<", re.IGNORECASE),
    'pics': re.compile(r"<\s*(?:Pics|pictures|Pictures|Pics)>\s*(.*?)\s*<", re.IGNORECASE),
    'response': re.compile(r"<\s*(?:Response|resp|Resp|response)>\s*(.*?)\s*<", re.IGNORECASE),
    'gmapID': re.compile(r"<\s*(?:gmapID|Gmap_id|gmap_id|GmapID|Gmap_id)\s*>\s*([\w:]+(?:-\d+)?(?:[A-Za-z0-9_-]+)?)\s*<", re.IGNORECASE)
}

This dictionary contains regular expressions (regex) that are used to find specific information within text data.<br>

user_id: This regex finds different ways of writing "user ID" and extracts the number that follows.<br>
name: This regex finds different ways of writing "name" and extracts the name that follows.<br>
time: This regex finds different ways of writing "time" and extracts the time information.<br>
rating: This regex finds different ways of writing "rating" and extracts the number that follows.<br>
text: This regex finds the main text content of the review.<br>
pics: This regex finds information related to pictures or images.<br>
response: This regex finds different ways of writing "response" and extracts the response information.<br>
gmapID: This regex finds Google Map IDs in different formats.<br>

## Defining helper functions

In [13]:
# Helper function to clean up extracted data.
def data_cleaning(data):
    """
    Description: Function removes the leading and trailing extra whitespaces to clean the input string
    Arguments: Data(str) on which cleaning operation needs to be done
    Returns: If data is there then cleaned data(str) or else if data is None then return None
    """
    return data.strip() if data else None

# Function to filter emojis from text
def remove_emoticons(text):
    """
    Description: Function is used to find and remove the emojis that falls within the defined range of unicodes from the input string
    Arguments: text(str), the txt string from which the emojis needs to be removed
    Return: str or None, string in which the defined emojis are removed, else None if the text input is None
    """
    if text is None:
        return None
    emoticons_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "]+",
        flags=re.UNICODE,
    )
    return emoticons_pattern.sub(r'', text)

# Function to extract image dimensions from URLs.
def parse_dimensions_from_url(url):
    """
    Description: The function reads and extracts the dimensions of the pictures from the url if given in the input files in the format 'w(\d+)-h(\d+), url' and
                returns the dimensions if the defined pattern is matched
    Arguments: url(str), URL or link from which the dimension of the image needs to be extracted
    Return: A list of integers containing height and width if the pattern is matched, else an empty list
    """
    match = re.search(r'w(\d+)-h(\d+)', url)
    if match:
        return [int(match.group(1)), int(match.group(2))]
    return []

# Extract image dimensions from text and handle different formats.
def get_image_dimensions(pics_text):
    """
    Description: This function is used to check if the picture dimensions are contained in a list or in dictionary of URLs. If the dimensions are present
                in a list format then the dimensions are appended to the defined 'dimensions' empty list, if dimensions are in format of dictionary of URLs
                then 'parse_dimensions_from_url()' function is invoked and dimensions are extracted from the URLs
    Arguments: pics_text(str), The string literal which contains the picture dimensions
    Return: A list of lists typically containing the dimensions of pictures that are height and width, else an empty list if no dimensions are found
    """
    dimensions = []
    if pics_text and pics_text != 'None':
        try:
            pics_data = eval(pics_text)
            for pic in pics_data:
                if isinstance(pic, list):
                    dimensions.append([int(dim) for dim in pic])  # Convert dimensions to integers
                elif isinstance(pic, dict) and 'url' in pic:
                    for url in pic['url']:
                        dim = parse_dimensions_from_url(url)
                        if dim:
                            dimensions.append(dim)
        except Exception as e:
            print(f"Error parsing pics_text: {pics_text}, error: {e}")
    return dimensions

Helper Functions:<br>

These functions are used to prepare the data for analysis.<br>

data_cleaning(): This function removes extra spaces from the beginning and end of text, making sure the data is consistent and clean.<br>
remove_emoticons(): This function takes out emojis from text using a special pattern, making the text more standard and preventing unwanted noise in the analysis.<br>
parse_dimensions_from_url(): This function gets the width and height of an image from a URL that follows a specific pattern. It returns these numbers as a list.<br>
get_image_dimensions(): This function finds and processes the width and height of images from text data. It can handle both lists and dictionaries of URLs. This function combines image dimension data for further analysis.<br>

## Functions to preprocess data from input text files and .xlsx file

In [14]:
# Extract relevant data from text content from each file and organize it into a list of dictionaries
def retrieve_data_from_txt(content):
    """
    Description: This function iterates through each input file and reads the content present between '<record>' and '</record>' tags treating each one of
                them as individual record. It then uses the defined regex pattern to extract the data for the specific fields from each record found.
                The data which is extracted is then stored in a dictionary and then further processed into a list of records
    Arguments: content(str), content of the input .txt files which contains the records in a semi structured format that need processing and extraction
    Return: A list of dictionary, and each dictionary represents data of each single record that has been processed and extracted
    """
    records = []
    for match in re.finditer(r"<record>(.*?)</record>", content, re.DOTALL):
        record_content = match.group(1)
        record = {}
        for key, pattern in regex_patterns.items():
            match_field = pattern.search(record_content)
            if key == 'pics':
                pic_dims = get_image_dimensions(match_field.group(1)) if match_field else []
                record['pic_dims'] = pic_dims
            elif key == 'response':
                response_value = data_cleaning(match_field.group(1)) if match_field else None
                # Only increment response count if response is not None or empty
                record['response'] = response_value if response_value else None
            else:
                record[key] = data_cleaning(match_field.group(1)) if match_field else None
        records.append(record)
    return records

# Function to Clean Excel data by dropping empty rows and irrelevant columns
def process_excel_data(df):
    """
    Description: Function is used to perform cleaning operation on .xlsx input file
                1. It will drop the rows which consists of NaN values for all the variables in that respective row
                2. It wll drop the pre-defined columns which are not necessary for processing and extraction process. Also, if no such pre defined columns are
                    found then the errors would be ignored
    Arguments: df(Dataframe), dataframe that has been loaded from the input .xlsx data which needs cleaning
    Return: df(Dataframe), the cleaned and processed dataframe
    """
    # Drop rows where all elements are NaN
    df = df.dropna(how='all')
    # Drop unnecessary columns
    df = df.drop(columns=['x0', 'x1', 'x2', 'x3', 'x4'], errors='ignore')
    return df

'retrieve_data_from_txt()': This function processes semi-structured text data by extracting relevant information based on predefined regex patterns. It iterates through records, captures specific fields like user IDs, ratings, and image dimensions, and compiles this information into a structured format (list of dictionaries).<br>

'process_excel_data()': This function cleans the loaded Excel data by removing rows with all missing values and dropping unnecessary columns. This ensures that only relevant and clean data remains for further analysis.<br>

### Reading, Parsing, and Extracting Data from Text Files into a DataFrame

In [15]:
# Inititlaise a list to hold the extracted records
all_records = []

# Read and parse each file
"""
Description: This code is responsible for reading and parsing each file in input_file_path list. It opean and read each file content and then calls the
             retrieve_data_from_txt function to extract the data from the content. Thaty data is stored in all_record list, if this function sucessfully
             processes the file then the first and last record of extracted data are printed else no rercords werer extracted are printed.
"""
# Iterate through each text file, read the content, extract data, and append it to all_records
for file_path in input_file_paths:
    if os.path.exists(file_path):  # Check if the file exists before trying to open it
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            extracted_records = retrieve_data_from_txt(content)
            all_records.extend(extracted_records)
            if extracted_records:
                first_record = extracted_records[0]
                last_record = extracted_records[-1]
            else:
                print(f"{file_path}: No records extracted")
    else:
        print(f"File does not exist: {file_path}")

# Convert extracted records into a DataFrame for further processing
df_txt = pd.DataFrame(all_records)

This part of the code looks at each text file in a list of file paths. It checks if the file exists, then uses a function called retrieve_data_from_txt() to get information from the file and puts it in a list called all_records. After looking at all the files, the collected data is turned into a table called df_txt for further analysis. This makes sure that the text data is gathered in a structured way and ready for analysis.

## Cleaning and preprocessing data

In [16]:
# Convert time-related data from numeric format to datetime format
df_txt['time'] = pd.to_numeric(df_txt['time'], errors='coerce')
df_txt['time'] = pd.to_datetime(df_txt['time'], unit='ms', errors='coerce')

# Convert text fields to lowercase and remove any emojis
df_txt['text'] = df_txt['text'].str.lower().apply(remove_emoticons)
df_txt['response'] = df_txt['response'].str.lower().apply(remove_emoticons)

# Load and combine all sheets from the Excel file into a single DataFrame and clean it
df_all_sheets_from_excel = pd.read_excel(excel_file_path, sheet_name=None)
excel_dataframe = pd.concat(df_all_sheets_from_excel.values(), ignore_index=True)
excel_dataframe = process_excel_data(excel_dataframe)

# Ensure consistency in column names for merging purposes
excel_dataframe = excel_dataframe.rename(columns={'gmap_id': 'gmapID'})

# Merge extracted text data and cleaned Excel data into a combined DataFrame
df_combined = pd.concat([excel_dataframe, df_txt], ignore_index=True)

This part of the code cleans and prepares the data that was extracted:<br>

Timestamp Conversion: The code changes the "time" information from numbers to a date and time format, which makes it easier to work with dates and times.<br>
Text Cleaning: It removes emojis from the "text" and "response" parts and makes all the letters lowercase, making the text analysis more consistent.<br>
Excel Data Loading and Cleaning: The data from the Excel file is loaded, cleaned by removing unwanted columns and empty rows, and then renamed to make sure the column names are the same.<br>
Data Merging: Finally, the cleaned Excel data is combined with the data from the text files into a single table called "df_combined" for a complete analysis, making sure all the important information is in one place.<br>

## Creating CSV output file

In [17]:
# Group data by 'gmapID' and calculate aggregated counts for reviews, texts, and responses
csv_output = df_combined.groupby('gmapID').agg(
    review_count=('user_id', 'count'),
    review_text_count=('text', lambda x: x.dropna().apply(lambda y: y if y.lower() != 'none' else None).dropna().ne('').sum()),
    response_count=('response', lambda x: x.dropna().apply(lambda y: y if y.strip().lower() != 'none' else None).dropna().ne('').sum())
).reset_index()

# Save the aggregated data to a CSV output file
csv_output.to_csv(csv_output_path, index=False)

This part of the code groups the data by Google Map ID and calculates important numbers:<br>

review_count: This counts how many reviews each Google Map ID has.<br>
review_text_count: This counts how many reviews are not empty and valid.<br>
response_count: This counts how many responses are not empty and valid.<br>
The grouped and calculated data is then saved as a CSV file in a specific location, making it ready for further analysis or reporting.<br>

## Creating JSON output file

In [18]:
# Replace NaN with None in the DataFrame for JSON export
df_combined = df_combined.where(pd.notnull(df_combined), None)

# Prepare and write the JSON output file
json_output = {}
for gmapID, group in df_combined.groupby('gmapID'):
    reviews = group.apply(lambda row: {
        'user_id': row['user_id'] if pd.notnull(row['user_id']) else 'None',
        'time': pd.to_datetime(row['time']).strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(row['time']) else 'None',
        'review_rating': row['rating'] if pd.notnull(row['rating']) else 'None',
        'review_text': row['text'] if pd.notnull(row['text']) else 'None',
        'If_pic': 'Y' if row['pic_dims'] else 'N',
        'pic_dim': row['pic_dims'] if row['pic_dims'] else [],
        'If_response': 'Y' if pd.notna(row['response']) and str(row['response']).strip().lower() != 'none' else 'N'
    }, axis=1).tolist()

    json_output[gmapID] = {
        'reviews': reviews,
        'earliest_review_date': pd.to_datetime(group['time'].min()).strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(group['time'].min()) else None,
        'latest_review_date': pd.to_datetime(group['time'].max()).strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(group['time'].max()) else None
    }
    
# Convert the data to a JSON string
json_str = json.dumps(json_output, ensure_ascii=False, indent=4)

# Custom replacement to format pic_dim specifically
json_str = re.sub(r'\[\n\s*(\d+),\s*\n\s*(\d+)\n\s*\]', r'["\1", "\2"]', json_str)

# Write the JSON string into a file
with open(json_output_path, 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)

This part of the code creates and saves the data in a JSON format:<br>

Handling Missing Values: Any missing values in the data are replaced with "None" so they can be used in the JSON format.<br>
Preparing JSON Data: For each Google Map ID, the code collects important review information like user ID, time, rating, review text, picture size, and response status. This information is organized into a dictionary. The earliest and latest review dates for each Google Map ID are also found.<br>
Formatting and Saving: The organized data is turned into a JSON string, with a special format for picture sizes. This JSON string is then saved to a file, creating a well-structured JSON output that can be easily shared or used for more work.

<div class="alert alert-block alert-success"> 

## References <a class="anchor" name="Ref"></a>

* Python Software Foundation. (n.d.). os.path — Common pathname manipulations. Python Documentation. https://docs.python.org/3/library/os.path.html
* Stack Overflow. (2024, February 23). Iteration over rows and apply function based on condition. https://stackoverflow.com/questions/78930375/iteration-over-rows-and-apply-function-based-on-condition

## Acknowledgement
* We acknowledge the assistance of ChatGPT, powered by OpenAI, in completing certain parts of this assignment. The use of this AI tool provided valuable support in areas such as   regex patterns, text preprocessing, and enhancing the overall quality of the work. 
OpenAI. (2023). ChatGPT (GPT-4). https://openai.com/chatgpt


</div>